In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import glob
import shutil
import os

In [2]:
pattern = 'latamSatData/datasetRGB_rescaled/*/'
fileStrings = glob.glob(pattern)


In [3]:
fileStrings

['latamSatData/datasetRGB_rescaled/all/']

In [4]:
classDictionary = {30: 'Herbaceous Vegetation',
20: 'Shrubs',
40: 'Agricultural Land',
50: 'Urban Areas',
60: 'Bare Earth/Sparse Vegetation',
70: 'Snow and Ice',
80: 'Permanent Water Bodies',
90: 'Herbaceous Wetland',
100: 'Moss and Lichen',
111: 'Closed Evergreen Needle Leaf Forest',
112: 'Closed Evergreen Broad Leaf Forest',
113: 'Closed Deciduous Needle Leaft Forest',
114: 'Closed Deciduous Broad Leaf Forest',
115: 'Closed Mixed Forest',
116: 'Other Closed Forest',
121: 'Open Evergreen Needle Leaf Forest',
122: 'Open Evergreen Broad Leaf Forest',
123: 'Open Deciduous Needle Leaft Forest',
124: 'Open Deciduous Broad Leaf Forest',
125: 'Open Mixed Forest',
126: 'Other Open Forest',
200: 'Oceans, Seas'}

#for idx, fs in enumerate(fileStrings):
  #  parts = fs.split('/')
  #  ecoregionCode = parts[-3]
  #  filename = ecoregionCode + '_' + parts[-1] 
  #  parts[-1] = filename
  #  parts[1] = topFolder
  #  parts.pop(2)
  #  className = classDictionary[int(parts[-2])]
  #  parts[-2] = className
 #   newFile = '/'.join(parts)
 #   newDir = '/'.join(parts[:-1])
#    if not os.path.exists(n#ewDir):
#        os.makedirs(newDir)#

#    shutil.copy(fs, newFile)
#    if idx % 1000 == 0 :
#        print(idx, 'done')
    
        
    
    
    

In [5]:
classes = glob.glob('latamSatData/datasetRGB_relabel/*/*')
classes = [c.split('/')[-1] for c in classes]
idxKeys = [i for i in range(len(classes))]

In [6]:
classnameDict = dict(zip(idxKeys, classes))

In [7]:
classnameDict

{0: 'Open Evergreen Broad Leaf Forest',
 1: 'Bare Earth Sparse Vegetation',
 2: 'Permanent Water Bodies',
 3: 'Closed Deciduous Broad Leaf Forest',
 4: 'Open Mixed Forest',
 5: 'Urban Areas',
 6: 'Snow and Ice',
 7: 'Oceans, Seas',
 8: 'Closed Evergreen Needle Leaf Forest',
 9: 'Agricultural Land',
 10: 'Other Closed Forest',
 11: 'Open Evergreen Needle Leaf Forest',
 12: 'Shrubs',
 13: 'Other Open Forest',
 14: 'Closed Mixed Forest',
 15: 'Open Deciduous Broad Leaf Forest',
 16: 'Closed Evergreen Broad Leaf Forest',
 17: 'Herbaceous Vegetation',
 18: 'Herbaceous Wetland'}

In [8]:
folder = 'latamSatData/datasetRGB_rescaled/'

builder = tfds.folder_dataset.ImageFolder(folder)
base_dataset = builder.as_dataset()['all']
filenames = []
labels = []
def onehot_encode(x):
    x['label'] = tf.one_hot(x['label'], 19)
    return x
base_dataset = base_dataset.map(onehot_encode)
base_dataset = base_dataset.shuffle(base_dataset.cardinality())
base_dataset = base_dataset.batch(32)





2023-10-25 19:58:14.501505: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
2023-10-25 19:58:17.173502: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2023-10-25 19:58:17.173528: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-25 19:58:17.173533: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-25 19:58:17.173566: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been b

In [9]:
def train_test_split_(dataset, test_size):
    dataset = dataset.shuffle(dataset.cardinality())
    pct_size = dataset.cardinality().numpy() / 100 
    test_size = round(pct_size * test_size)
    test_dataset = dataset.take(test_size)
    train_dataset = dataset.skip(test_size)
    return train_dataset, test_dataset

In [10]:
train, test = train_test_split_(base_dataset, 20)

In [11]:
def make_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Entry block
    x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(128, 3, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 768]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(size, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(size, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = tf.keras.layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = tf.keras.layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(units, activation=activation)(x)
    return tf.keras.Model(inputs, outputs)


m = make_model(input_shape=(64,64,3), num_classes=19)

In [12]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="rgb_logscale_cleanlab_classification",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [13]:

m.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
    run_eagerly=False
)

In [22]:
def get_xy_training(x, y):
    return x, y
train = train.map(get_xy_training)
test = train.map(get_xy_training)

In [24]:
train_history = m.fit(train, epochs=36, validation_data=test, callbacks=[model_checkpoint_callback])


Epoch 1/36


2023-10-25 19:59:45.315961: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-25 19:59:55.404741: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 216095 of 298857
2023-10-25 19:59:59.094695: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.
2023-10-25 19:59:59.096697: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 1 of 9340
2023-10-25 20:00:04.235619: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] Shuffle buffer filled.


 886/7472 [==>...........................] - ETA: 5:11 - loss: 2.2801 - accuracy: 0.2443

KeyboardInterrupt: 

In [ ]:
break

In [ ]:
m = tf.keras.models.load_model('base_model_for_cleanlab_v2')

In [ ]:
feature_extraction = tf.keras.Model(m.input, m.layers[-3].output)


In [ ]:
class_to_int_dict = dict(zip(classnameDict.values(), classnameDict.keys()))
class_to_int_dict

In [ ]:
from cleanlab import Datalab


In [ ]:
from PIL import Image
import numpy as np

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
issues_list = []
labs_list = []
filepaths = []
i = 1
for batch in base_dataset:
    print('batch', i)
    images = batch['image'].numpy()
    classes = batch['label'].numpy()
    paths = batch['image/filename']
    filepaths.append(paths)
    classProb = m.predict(images, verbose=0)
    predFeat= feature_extraction.predict(images, verbose=0)
    data_to_clean = {'Images':np.squeeze(np.array(images)), 'Labels':np.argmax(classes, axis=1)}
    lab = Datalab(data=data_to_clean, label_name="Labels", image_key="Images")
    lab.find_issues(pred_probs=np.squeeze(np.array(classProb)), features=np.squeeze(np.array(predFeat)))
    issues = lab.get_issues('label')
    issues['filepath'] = paths
    issues_list.append(issues)
    labs_list.append(lab)
    i += 1 
    


In [ ]:
all_issues = pd.concat(issues_list)


In [ ]:
all_issues
all_issues['given_label'] = all_issues['given_label'].map(classnameDict)
all_issues['predicted_label'] = all_issues['predicted_label'].map(classnameDict)


In [ ]:
all_issues['filepath'] = all_issues['filepath'].apply(lambda x: x.decode())

In [ ]:
all_issues[all_issues['is_label_issue']]

def relabel_filepath(file_path, new_label):
    file_path = file_path.split('/')
    file_path[-2]=new_label
    file_path = '/'.join(file_path)
    return file_path
all_issues['new_filepath'] = all_issues.apply(lambda x: relabel_filepath(x.filepath, x.predicted_label), axis=1)



In [ ]:

def move_file(old_path, new_path):
    shutil.move(old_path, new_path)  


In [ ]:
all_issues

In [ ]:
all_issues

In [ ]:
all_issues.to_csv('relabel_1.csv')

In [ ]:
def relabel_images(filepaths, issues, 

In [ ]:
issues_list = []
for i in range(len(labs_list)):
    label_issues = labs_list[i].get_issues("label")
    label_issues = label_issues[label_issues['is_label_issue']]
    label_issues['given_label'] = label_issues['given_label'].map(classnameDict)
    label_issues['predicted_label'] = label_issues['predicted_label'].map(classnameDict)
    issues_list.append(label_issues)
label_issues = pd.concat(issues_list)

In [ ]:
label_issues.columns

In [ ]:
import pandas as pd

In [ ]:
adj

In [ ]:
label_issues = all_issues[all_issues['is_label_issue']]
adj = pd.crosstab(label_issues['given_label'], label_issues['predicted_label'])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(adj)
plt.xticks(range(len(adj.columns)), adj.columns,rotation='vertical')
plt.ylabel('Given Label')
plt.xlabel('New Label')
plt.yticks(range(len(adj.index)), adj.index)
plt.colorbar()
plt.show()

In [ ]:
issues = label_issues[label_issues['is_label_issue']]

In [ ]:
issues['given_label']